# Проект для «Викишоп»

В данной проекте мной решалась задача классификации

На основе градиентного бустинга была построена модель, которая определяет, является ли комментарий позитивным или негативным

## Подготовка <a id="section1"></a>

### Импорт библиотек

In [1]:
!pip install spacy
!pip install spacy download en

  Using cached download-0.3.5-py3-none-any.whl (8.8 kB)


ERROR: Could not find a version that satisfies the requirement en (from versions: none)
ERROR: No matching distribution found for en


In [2]:
from tqdm import tqdm
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')
tqdm.pandas()

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import (train_test_split, 
                                     GridSearchCV,
                                     StratifiedKFold)
from sklearn.linear_model import RidgeClassifier, LogisticRegression
from lightgbm import LGBMClassifier

from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
import spacy
from pymystem3 import Mystem

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Кимон\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Кимон\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
RANDOM_STATE = 666

### Загрузка данных

In [4]:
try:
    data = pd.read_csv('../datasets/toxic_comments.csv1')
except:
    pass

### Краткий анализ текста

In [5]:
data = data.drop_duplicates()

In [6]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Мы видим, что есть два столбца. Текст и оценка

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [8]:
data['toxic'].value_counts(normalize = True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

Мы видим что классы несбалансированы. Но мы будем избавляться от дисбаланса путём передачи нужного аргумента в модель

### Лемматизация

Создадим функцию с лемматизацией

In [9]:
nlp = spacy.load("en_core_web_sm")

def lemmatize(text):
    doc = nlp(text)
    # Extract the lemma for each token and join
    return " ".join([token.lemma_ for token in doc])

def clear_text(text):
    text = re.sub(r"[^a-zA-Z']", ' ', text) 
    text = " ".join(text.split())
    return text



Посмотрим несколько предложений до лемматизации

In [10]:
%%time

data['text'] = data['text'].str.lower()
data['text'] = data['text'].apply(clear_text)
data['lemm_text'] = data['text'].progress_apply(lemmatize)

100%|██████████████████████████████████████████████████████████████████████████| 159292/159292 [32:13<00:00, 82.38it/s]

CPU times: total: 32min 19s
Wall time: 32min 17s


In [11]:
data['lemm_text']  = data['lemm_text'].values.astype('U')

In [12]:
list(data.iloc[0])

["explanation why the edits made under my username hardcore metallica fan were reverted they weren't vandalisms just closure on some gas after i voted at new york dolls fac and please don't remove the template from the talk page since i'm retired now",
 0,
 'explanation why the edit make under my username hardcore metallica fan be revert they be not vandalism just closure on some gas after I vote at new york dolls fac and please do not remove the template from the talk page since I be retire now']

Итого лемматизация прошла успешно

### Разделение на обучающую и тестовые выборки

Будем пользоваться кросс-валидацией, так что возьмём только 10 % данных для теста

In [13]:
feature = data[['lemm_text']]
target = data['toxic']

In [14]:
# передаём stratify, так как классы несбалансированы
feature_train, feature_test, target_train, target_test = train_test_split(
        feature, target, test_size=0.1, stratify = target
)

### Создание Pipeline

In [15]:
# создание Pipeline, который трансформирует текст
text_transformer = Pipeline(steps=[
    #('lemma', FunctionTransformer(lemmatize_df)),
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
])

# создание Columntransform, чтобы применить преобразование к одному столбцу
preprocessor = ColumnTransformer(
    transformers=[
        ('text_transformer', text_transformer, 'lemm_text'),
    ])


## Обучение моделей

Так как классы несбалансированы будет передавать StratifiedKFold в cv

In [16]:
skf = StratifiedKFold(n_splits=3)

### Линейная регрессия

In [17]:
rc = LogisticRegression(random_state = RANDOM_STATE,
                    class_weight = 'balanced')

steps = [
    ('preprocessor', preprocessor), 
    ('rc', rc)
]

pipe_rc = Pipeline(steps)

In [18]:
# params = {
#     'rc__alpha' : [0.1, 0.5, 1, 10]
# }

In [19]:
grid_rc = GridSearchCV(pipe_rc,
                      param_grid ={},
                      cv = skf,
                      scoring='f1',
                      verbose=2)

In [20]:
%%time

grid_rc.fit(feature_train, target_train) 
grid_rc.best_score_

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END .................................................... total time=  12.4s
[CV] END .................................................... total time=   9.6s
[CV] END .................................................... total time=   9.8s
CPU times: total: 40.5 s
Wall time: 43.7 s


0.7499298596338758

---
Итого f1 = 0.75. Модель логистической регрессии прошла проверку

---

### LightGBM

In [21]:
lg = LGBMClassifier(random_state = RANDOM_STATE)

In [22]:
steps = [
    ('preprocessor', preprocessor), 
    ('lg', lg)
]
pipe_lg = Pipeline(steps)

In [23]:
grid_lg = GridSearchCV(pipe_lg,
                      param_grid ={},
                      cv = skf,
                      scoring='f1',
                      verbose=2)

In [24]:
%%time

grid_lg.fit(feature_train, target_train) 
grid_lg.best_score_

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END .................................................... total time=  27.9s
[CV] END .................................................... total time=  27.0s
[CV] END .................................................... total time=  27.2s
CPU times: total: 11min 57s
Wall time: 2min 1s


0.7523612765060813

Итак нужная метрика достигнута

### Проверка лучшей модели

In [25]:
grid_lg.score(feature_test, target_test)

0.7561327561327562

Модель подтвердила свою эффективность

## Выводы

 - Нужной метрики удалось добиться на LightGBM. В целом модель достаточно быстро обучается и лучше вариант сложно будет найти
 - Линейная регрессия показала себя плохо. Скорее всего стоило брать модель без регуляризации, тогда бы показатели могли быть лучше
 - Лемматизацию нельзя применять ко всему предложению, а надо применять только к словам. Сами предложения будут обрабатываться очень долго